In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.environ.get('OPENAI_API_KEY')
os.environ['ACTIVELOOP_TOKEN'] = os.environ.get('ACTIVELOOP_TOKEN')

# Calling

## Invoke

In [3]:
from langchain import PromptTemplate, LLMChain
from langchain_openai import OpenAI

prompt_template = "What is a word to replace the following: {word}?"

# Set the "OPENAI_API_KEY" environment variable before running following line.
llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0)

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)
llm_chain.invoke("artificial")

{'word': 'artificial', 'text': '\n\nSynthetic'}

## Apply

In [4]:
input_list = [
    {"word": "artificial"},
    {"word": "intelligence"},
    {"word": "robot"}
]

llm_chain.apply(input_list)

[{'text': '\n\nSynthetic'},
 {'text': '\n\nCleverness'},
 {'text': '\n\nandroid'}]

## Generate

The .generate() method will return an instance of LLMResult, which provides more information. For example, the finish_reason key indicates the reason behind the stop of the generation process. It could be stopped, meaning the model decided to finish or reach the length limit. There is other self-explanatory information like the number of total used tokens or the used model.

In [5]:
llm_chain.generate(input_list)

LLMResult(generations=[[Generation(text='\n\nSynthetic', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nCleverness', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nandroid', generation_info={'finish_reason': 'stop', 'logprobs': None})]], llm_output={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 33, 'total_tokens': 42}, 'model_name': 'gpt-3.5-turbo-instruct'}, run=[RunInfo(run_id=UUID('ff1b7c75-53c4-4179-9d61-9a1230b376c2')), RunInfo(run_id=UUID('327d6a53-97d7-42da-a88a-9694e7e38e1d')), RunInfo(run_id=UUID('c09c2317-37db-4bf2-841a-c328eb58aec9'))])

## Predict

The next method we will discuss is .predict(). (which could be used interchangeably with .run()) Its best use case is to pass multiple inputs for a single prompt. However, it is possible to use it with one input variable as well. The following prompt will pass both the word we want a substitute for and the context the model must consider.

#### Multiple

In [11]:
prompt_template = "Looking at the context of '{context}'. What is a approapriate word to replace the following: {word}?"

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(template=prompt_template, input_variables=["word", "context"]))

llm_chain.predict(word="fan", context="house")

'\n\nVentilation system'

In [12]:
llm_chain.predict(word="fan", context="")

'\n\nSupporter'

In [ ]:
# llm_chain.run(word="fan", context="object")

'\n\nVentilator'

### from string

In [13]:
template = """Looking at the context of '{context}'. What is a approapriate word to replace the following: {word}?"""
llm_chain = LLMChain.from_string(llm=llm, template=template)

In [14]:
llm_chain.predict(word="fan", context="object")

'\n\nSupporter'

# Parsers

As discussed, the output parsers can define a data schema to generate correctly formatted responses. It wouldn’t be an end-to-end pipeline without using parsers to extract information from the LLM textual output. The following example shows the use of CommaSeparatedListOutputParser class with the PromptTemplate to ensure the results will be in a list format.

In [29]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
template = """List all possible words as substitute for 'artificial' as comma separated."""

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(template=template, input_variables=[]),
    output_parser=output_parser)

llm_chain.predict()

['synthetic',
 'man-made',
 'fake',
 'imitation',
 'simulated',
 'faux',
 'fabricated',
 'counterfeit',
 'ersatz',
 'false',
 'pretend',
 'mock',
 'spurious',
 'bogus',
 'phony',
 'contrived',
 'manufactured',
 'unnatural',
 'inauthentic']

# Conversational Chain

Depending on the application, memory is the next component that will complete a chain. LangChain provides a ConversationalChain to track previous prompts and responses using the ConversationalBufferMemory class.

In [33]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

output_parser = CommaSeparatedListOutputParser()
conversation = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory(),
    output_parser=output_parser
)

conversation.predict(input="List all possible words as substitute for 'artificial' as comma separated.")

['Synthetic',
 'simulated',
 'man-made',
 'fake',
 'imitation',
 'fabricated',
 'manufactured',
 'engineered',
 'virtual',
 'computer-generated',
 'digital',
 'non-natural',
 'inauthentic',
 'contrived',
 'ersatz',
 'pseudo',
 'phony',
 'false',
 'unreal',
 'pretend',
 'counterfeit',
 'spurious',
 'bogus',
 'sham',
 'feigned',
 'deceptive',
 'fraudulent',
 'specious',
 'fallacious',
 'misleading',
 'delusive',
 'illusory',
 'fictitious',
 'make-believe',
 'imaginary',
 'unreal',
 'fictive',
 'fanciful',
 'fantastic',
 'mythical',
 'legendary',
 'mythological',
 'fabricated',
 'created',
 'contrived',
 'invented',
 'made-up',
 'unreal',
 'imaginary',
 'fictional',
 'mythical',
 'legendary',
 'mythological',
 'fanciful',
 'fantastic',
 'make-believe',
 'pretend',
 'simulated',
 'virtual',
 'computer-generated',
 'digital',
 'non-natural',
 'inauthentic',
 'ersatz',
 'pseudo',
 'phony',
 'false',
 'counterfeit',
 'spurious',
 'bogus',
 'sham',
 'feigned',
 'deceptive',
 'fraudulent',
 'sp

In [34]:
conversation.predict(input="And the next 4?")

["['make-believe'", "'pretend'", "'simulated'", "'virtual']"]

# Debug

In [39]:
template = """List all possible words as substitute for 'artificial' as comma separated.

Current conversation:
{history}

{input}"""

conversation = ConversationChain(
    llm=llm,
    prompt=PromptTemplate(template=template, input_variables=["history", "input"], output_parser=output_parser),
    memory=ConversationBufferMemory(),
    verbose=True)

conversation.predict(input="")



> Entering new ConversationChain chain...
Prompt after formatting:
List all possible words as substitute for 'artificial' as comma separated.

Current conversation:




> Finished chain.


'\n1. Synthetic\n2. Man-made\n3. Faux\n4. Imitation\n5. Simulated\n6. Fake\n7. Replicated\n8. Manufactured\n9. Counterfeit\n10. Mock\n11. Pretend\n12. False\n13. Inauthentic\n14. Unnatural\n15. Fabricated'

# Sequential Chain

Another helpful feature is using a sequential chain that concatenates multiple chains into one. The following code shows a sample usage

In [35]:
# poet
poet_template: str = """You are an American poet, your job is to come up with\
poems based on a given theme.

Here is the theme you have been asked to generate a poem on:
{input}\
"""

poet_prompt_template: PromptTemplate = PromptTemplate(
    input_variables=["input"], template=poet_template)

# creating the poet chain
poet_chain: LLMChain = LLMChain(
    llm=llm, output_key="poem", prompt=poet_prompt_template)

# critic
critic_template: str = """You are a critic of poems, you are tasked\
to inspect the themes of poems. Identify whether the poem includes romantic expressions or descriptions of nature.

Your response should be in the following format, as a Python Dictionary.
poem: this should be the poem you received
Romantic_expressions: True or False
Nature_descriptions: True or False

Here is the poem submitted to you:
{poem}\
"""

critic_prompt_template: PromptTemplate = PromptTemplate(
    input_variables=["poem"], template=critic_template)

# creating the critic chain
critic_chain: LLMChain = LLMChain(
    llm=llm, output_key="critic_verified", prompt=critic_prompt_template)

In [36]:
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(chains=[poet_chain, critic_chain])

In [38]:
# Run the poet and critic chain with a specific theme
theme: str = "the beauty of nature"
review = overall_chain.invoke(theme)

# Print the review to see the critic's evaluation
print(review)

{'input': 'the beauty of nature', 'output': "\n\npoem: Nature's Beauty\nRomantic_expressions: False\nNature_descriptions: True"}


# Custom Chain

In [40]:
from langchain.chains import LLMChain
from langchain.chains.base import Chain

from typing import Dict, List


class ConcatenateChain(Chain):
    chain_1: LLMChain
    chain_2: LLMChain

    @property
    def input_keys(self) -> List[str]:
        # Union of the input keys of the two chains.
        all_input_vars = set(self.chain_1.input_keys).union(set(self.chain_2.input_keys))
        return list(all_input_vars)

    @property
    def output_keys(self) -> List[str]:
        return ['concat_output']

    def _call(self, inputs: Dict[str, str]) -> Dict[str, str]:
        output_1 = self.chain_1.run(inputs)
        output_2 = self.chain_2.run(inputs)
        return {'concat_output': output_1 + output_2}

In [41]:
prompt_1 = PromptTemplate(
    input_variables=["word"],
    template="What is the meaning of the following word '{word}'?",
)
chain_1 = LLMChain(llm=llm, prompt=prompt_1)

prompt_2 = PromptTemplate(
    input_variables=["word"],
    template="What is a word to replace the following: {word}?",
)
chain_2 = LLMChain(llm=llm, prompt=prompt_2)

concat_chain = ConcatenateChain(chain_1=chain_1, chain_2=chain_2)
concat_output = concat_chain.run("artificial")
print(f"Concatenated output:\n{concat_output}")

Concatenated output:


Artificial means something that is made or produced by humans, rather than occurring naturally. It can also refer to something that is not genuine or authentic, but rather created or imitated to appear real.

Synthetic
